In [6]:
# Simple function that takes an rsid as input, sends a curl request to LdLink, filters the output for SNPs
# with R2>0.8 and outputs a text file with all SNPs sorted by R2 value

import requests
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def ldproxy(rsid, pop = 'CEU+FIN+GBR+TSI+IBS', threshhold=0.8):
    
    #rsid - rsid you are interested in
    #R2 threshhold - 0.8 by default 
    
    token = 'da0eb217dded'   # this is the token I got for API access can be different for other users
     #these populations were selected for closeness to Nordic population - can also be altered
    params = (
        ('var', rsid),
        ('pop', pop),
        ('r2_d', 'r2'),
        ('token', token),
        ('genome_build', 'grch38_high_coverage')
    )
    response = requests.get('https://ldlink.nci.nih.gov/LDlinkRest/ldproxy', params=params, verify=False)
    
    # the requests library produces a response object that must be first converted into a single string
    # and then split by tabs and newlines
    
    inputlist=[]
    
    for row in response:
        inputlist.append(row)
    
    inputlist = [row.decode('UTF-8') for row in inputlist]
    x = inputlist[0]
    for i in range(1,len(inputlist)):
        x = x +inputlist[i]
    x_split = x.split('\n')
    for i in range(len(x_split)):
        x_split[i] = x_split[i].split('\t')
    df = pd.DataFrame(x_split)
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    df = df.drop(df.index[-1])
    df.R2 = df.R2.astype(float)
    df = df[df.R2>threshhold]
    return df
    
    

In [7]:
getproxy('rs11723120')

,RS_Number,Coord,Alleles,MAF,Distance,Dprime,R2,Correlated_Alleles,RegulomeDB,Function
1,rs59075460,chr4:102581394,(G/A),0.3588,0,1.0,1.0000,"G=G,A=A",7,NA
2,rs230545,chr4:102579897,(C/G),0.3588,-1497,1.0,1.0000,"G=G,A=C",5,NA
3,rs1598859,chr4:102585287,(T/C),0.3588,3893,1.0,1.0000,"G=T,A=C",1d,NA
4,rs17032850,chr4:102586546,(G/C),0.3588,5152,1.0,1.0000,"G=G,A=C",2b,NA
5,rs4648045,chr4:102587546,(T/C),0.3588,6152,1.0,1.0000,"G=T,A=C",6,NA
...,...,...,...,...,...,...,...,...,...,...
72,rs4648050,chr4:102593584,(T/C),0.33,12190,0.9624,0.8152,"G=T,A=C",6,NA
73,rs61316972,chr4:102608806,(A/G),0.3131,27412,1.0,0.8145,"G=A,A=G",7,NA
74,rs4699031,chr4:102605271,(A/G),0.3151,23877,0.9951,0.8140,"G=A,A=G",7,NA
75,rs1578821150,chr4:102603381,(-/T),0.3141,21987,0.9951,0.8102,"G=-,A=T",.,NA
